In [8]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient('localhost', 27018)
db = client.TableView
collection = db["dbSNP153_Annotation"]

In [9]:
db.collection_names()

<ipython-input-9-8ff4d2505e25>:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  db.collection_names()


['gene',
 'Promoter',
 'Disease2SNP',
 'gene2snps',
 'bin_NHCFV',
 'dbSNP153_Annotation']

In [4]:
df = pd.read_csv("df_gwas_collection.csv")

df = df[df.columns[1:]]

In [5]:
df.head()

,RSID,trait,mapped_gene
0,rs141175086,Morning vs. evening chronotype,LINC01128
1,rs2272756,Skin reflectance (Melanin index),NR
2,rs115438739,California verbal learning test score,NOC2L
3,rs13303010,Pancreatic cancer,NOC2L
4,rs13303327,Heel bone mineral density,KLHL17


In [28]:
df = df.drop_duplicates()

In [29]:
df.shape

(154842, 3)

In [30]:
len(df["RSID"].unique())

111492

In [31]:
df_mapped_genes = df[["RSID", "mapped_gene"]].drop_duplicates()

In [32]:
df_mapped_genes.shape

(128336, 2)

In [54]:
import collections
rsid_dict = collections.defaultdict(list)
for i, r in df_mapped_genes.iterrows():
    if pd.isna(r.mapped_gene) or r.mapped_gene == "NR":
        continue
    if "," in r.mapped_gene:
        mapped_gene = [x.strip() for x in r.mapped_gene.split(",")]
    else :
        mapped_gene = [r.mapped_gene]
    rsid_dict[r.RSID] = list(set(rsid_dict[r.RSID]) | set(mapped_gene))

In [55]:
dup_rsid = [ key for key in rsid_dict if len(rsid_dict[key]) > 1]

In [56]:
len(dup_rsid)

13082

In [57]:
dup_rsid[:10]

['rs75972122',
 'rs6603785',
 'rs12103',
 'rs1886773',
 'rs71628956',
 'rs78389809',
 'rs3765908',
 'rs139385870',
 'rs2803328',
 'rs2843152']

In [58]:
rsid_dict["rs35675666"]

['PARK7', 'ERFFI1', 'UTS2', 'TNFRSF9']

In [59]:

def update(rsid_dict, verbose=False, generateFile=False):
    def addDataIfExist(r, label):
        if not pd.isna(r[label]):
            return r[label]
        else:
            return ""
    for rsid in rsid_dict:
        collection.update_many({"RSID": rsid}, {"$set": { "gwas_gene": rsid_dict[rsid]}})
update(rsid_dict)

In [11]:
for r in collection.find({"RSID": "rs78389809"}):
    print(r)

In [10]:
import pprint
db = client.TableView
collection = db["dbSNP153_Annotation"]
for doc in collection.find({"RSID": "rs6910233" }):
    pprint.pprint(doc)

{'AAChange_Ensembl': '',
 'Alt': 'A',
 'Chr': 6,
 'End': 33566949,
 'GeneInfo_DistNG_Ensembl': 'dist=3819;dist=5598',
 'GeneName_ID_Ensembl': 'Z93017.1,BAK1',
 'NHCFV': {'ATAC': {'q_val(-log10)': 22.05603,
                    'region': 'chr6:33566606-33567358'},
           'ChromHMM': '25_Quies',
           'Fantom5': '',
           'Footprinting_MPBS': [],
           'HomerSig_Promoter_Regulaotry': '',
           'HomerSig_Regulaotry_Promoter': 'RegulatoryBin:chr6:33566000:33568000,RegulatoryID:interaction83980_chr6_bin1,PromoterBin:chr6:33432000:33434000,PromoterID:interaction83980_chr6_bin2,InteractionReads:6.000000,LogP:-7.437166,FDR:1.000000,TargetGene:Gene:SYNGAP1,Transcript:ENST00000645250.1,TxtStart:33431910,Gene:SYNGAP1,Transcript:ENST00000428982.4,TxtStart:33431730'},
 'RSID': 'rs6910233',
 'Ref': 'G',
 'Region': 'intergenic',
 'Start': 33566949,
 '_id': ObjectId('5eccc2682a86ac942dcd3844'),
 'gwasCatalog': 'Trait=Chagas cardiomyopathy in Tripanosoma cruzi '
                '

In [6]:
db = client.TableView
collection = db["dbSNP153_Annotation"]
for doc in collection.find({ "gwas_gene": { '$exists': True } }):
    print(doc)

In [66]:
for doc in collection.find({"RSID": { "$in": [ "rs2843152", "rs1414996067" ] }}):
    print(doc)

{'_id': ObjectId('5ecdb090b1ef59e0d227b0ae'), 'RSID': 'rs1414996067', 'Chr': 6, 'Start': 150845451, 'End': 150845453, 'Ref': 'CTG', 'Alt': 'C', 'Region': 'intergenic', 'GeneInfo_DistNG_Ensembl': 'dist=1786;dist=20096', 'NHCFV': {'ChromHMM': '8_TxWk', 'Fantom5': '', 'ATAC': '', 'Footprinting_MPBS': '', 'HomerSig_Regulaotry_Promoter': '', 'HomerSig_Promoter_Regulaotry': ''}, 'GeneName_ID_Ensembl': 'PLEKHG1,MTHFD1L', 'AAChange_Ensembl': '', 'gwasCatalog': ''}


In [72]:
## test
client = MongoClient('localhost', 27017)
db = client.test
collection = db.Annotation_test

In [76]:
for doc in collection.find({"RSID": "rs1414996067"}):
    print(doc)

{'_id': ObjectId('5ecdaedbb1ef59e0d2233217'), 'RSID': 'rs1414996067', 'Chr': 6, 'Start': 150845451, 'End': 150845453, 'Ref': 'CTG', 'Alt': 'C', 'Region': 'intergenic', 'GeneInfo_DistNG_Ensembl': 'dist=1786;dist=20096', 'NHCFV': {'ChromHMM': '8_TxWk', 'Fantom5': '', 'ATAC': '', 'Footprinting_MPBS': '', 'HomerSig_Regulaotry_Promoter': '', 'HomerSig_Promoter_Regulaotry': ''}, 'GeneName_ID_Ensembl': 'PLEKHG1,MTHFD1L', 'AAChange_Ensembl': '', 'gwasCatalog': ''}


In [77]:
collection.update_many({"RSID": "rs1414996067"}, {"$set":{"gwas_gene":"test"}})

In [79]:
for doc in collection.find({ "gwas_gene": { '$exists': True } }):
    print(doc)

{'_id': ObjectId('5ecdaedbb1ef59e0d2233217'), 'RSID': 'rs1414996067', 'Chr': 6, 'Start': 150845451, 'End': 150845453, 'Ref': 'CTG', 'Alt': 'C', 'Region': 'intergenic', 'GeneInfo_DistNG_Ensembl': 'dist=1786;dist=20096', 'NHCFV': {'ChromHMM': '8_TxWk', 'Fantom5': '', 'ATAC': '', 'Footprinting_MPBS': '', 'HomerSig_Regulaotry_Promoter': '', 'HomerSig_Promoter_Regulaotry': ''}, 'GeneName_ID_Ensembl': 'PLEKHG1,MTHFD1L', 'AAChange_Ensembl': '', 'gwasCatalog': '', 'gwas_gene': 'test'}


In [83]:
rsid_dict["rs6910233"]

[]

In [85]:
df[df["RSID"].str.contains("rs6910233")]

,RSID,trait,mapped_gene
149659,rs6910233,Chagas cardiomyopathy in Tripanosoma cruzi ser...,NR
